In [ ]:
# Before launching - setup new conda enviroment with following:
# $> conda create --name mllm -c conda-forge ipykernel ipywidgets tqdm tensorflow
# Prerequizites
%pip install nltk
%pip install pandas
%pip install transformers
%pip install pymorphy2

In [1]:
# Stage 0: Dataset loading and preprocessing

################################## IMPORTS ###################################
# First we import standard lib modules
#import os      -  slightly outdated, there is a better way since 3.4
from pathlib import Path                    # abstract cross-platform path
from shutil import unpack_archive           # high-level shell-functionality
import string                               # stdlib string support

# Next after blank line - pip/conda packages - not a standard library
from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm
import pandas as pd

# Lastly our own project modules - not very applicable to a single notebook:)


In [ ]:
# Stage 0: Dataset loading and preprocessing

############################## GLOBAL CONSTANTS ##############################
## Paths
WORK_DIR_PATH = Path('.')
DATASET_ZIP_PATH = WORK_DIR_PATH.joinpath('Articles_dataset_rus.zip')
INTERMID_DIR_PATH = WORK_DIR_PATH.joinpath('unzipped-dataset')
print(f'Dataset located in working dir: {DATASET_ZIP_PATH.is_file()}')

## Parameters
#PUNCTUATION_CHARS = {'(', ')', ':', ';', ',', '.', '"', '»', '«', '[', ']', '{', '}', '%'}
PUNCTUATION_CHARS = {char for char in string.punctuation}
print(f'Set of punctuation characters used:\n{PUNCTUATION_CHARS}')

PRETRAINED_MODEL_NAME = 'rubert-base-cased'
PRETRAINED_MODEL_OWNER = 'DeepPavlov'


In [ ]:
# Stage 0: Dataset loading and preprocessing

def tokenize(text: str) -> List[str]:
    """Our custom spin on the NLTK tokenizator
    
    NLTK wordpunct_tokenize(..) may return consequetive punctuation chars
    as a single token. We want to overwrite this behavior and have each as
    a separate token. Therefore validation and token list expantion is needed.
    """
    def _is_punctuation_only(token: str) -> bool:
        """Simple check if the token string consist of punctuation only
        
        This is not very elegant but rather efficient way to do this.
        First - build-in convertion of a string to set, which will leave
        only unique characters. Then simply checking with its method
        if the resulted set is a subset of our defined PUNCTUATION_CHARS.
        """
        return True if set(token).issubset(PUNCTUATION_CHARS) else False

    tokens = wordpunct_tokenize(text)       # Obtaining tokens via NLTK
    validated_tokens = []
    for token in tokens:
        if _is_punctuation_only(token):
            validated_tokens.extend(list(token))
        else:
            validated_tokens.append(token)
    return validated_tokens


def csv_tokens_from_txt(filepath: Path):
    """Read whole text file in a single string, tokenize and save as csv
    """
    with open(filepath, 'r', encoding='utf-8') as txtfile:
        # Easier way to read whole file in a long single-line string
        text = txtfile.read().replace('\n', ' ')
    # Pandas dataframe directly from the list returned by function
    data = pd.DataFrame(tokenize(text), columns=['token'])
    filename = f'{filepath.stem}.csv'
    data.to_csv(filepath.parent.joinpath(filename), index_label='id')


def parse_tokens(dataset_root: Path):
    """Parse tokens from subfolders of dataset root (text.txt -> text.csv)
    """
    for dirpath in tqdm(dataset_root.iterdir(), desc='Tokenization:'):
        if not dirpath.is_dir() or not dirpath.joinpath('text.txt').exists():
            continue        # Skip to the next iteration if not a subfolder
        csv_tokens_from_txt(dirpath.joinpath('text.txt'))
                

In [4]:
# Stage 1: Obtaining pretrained model
from transformers import BertConfig, TFBertForTokenClassification
import tensorflow as tf

def get_model():
    config = BertConfig.from_pretrained(
                PRETRAINED_MODEL_NAME, 
                from_pt = True, num_labels=3)  
    model = TFBertForTokenClassification.from_pretrained(
                f'{PRETRAINED_MODEL_OWNER}/{PRETRAINED_MODEL_NAME}',
                from_pt = True, config=config)
    model.layers[-1].activation = tf.keras.activations.softmax
    print(model.summary())
    return model

In [ ]:
################################## WORKFLOW ##################################
# Stage 0.
unpack_archive(DATASET_ZIP_PATH, INTERMID_DIR_PATH, 'zip')
dataset_root = INTERMID_DIR_PATH.joinpath(DATASET_ZIP_PATH.stem)
parse_tokens(dataset_root)
# Stage 1.
model = get_model()

# Stage 2.

In [ ]:
def validate_sequence(seq: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
    """ Валидация последовательности тэгов: убеждаемся, что первый токен каждого термина имеет тэг "B-TERM"

    :param seq: входная последовательность
    :return: провалидированная последовательность
    """
    is_previous_token = False
    validated_seq = []
    for token, tag in seq:
        if tag == Tags.I_TERM.value:
            if not is_previous_token:
                validated_seq.append((token, Tags.B_TERM.value))
            else:
                validated_seq.append((token, tag))
            is_previous_token = True
        elif tag == Tags.B_TERM.value:
            validated_seq.append((token, tag))
            is_previous_token = True
        else:
            validated_seq.append((token, tag))
            is_previous_token = False
    return validated_seq


In [ ]:
from enum import Enum


class Tags(Enum):
    B_TERM = 'B-TERM'
    I_TERM = 'I-TERM'
    NOT_TERM = 'O'


TERM_SET = {Tags.B_TERM.value, Tags.I_TERM.value}

label2class = {
            Tags.NOT_TERM.value: 0,
            Tags.B_TERM.value: 1,
            Tags.I_TERM.value: 2
        }

class2label = {
            0: Tags.NOT_TERM.value,
            1: Tags.B_TERM.value,
            2: Tags.I_TERM.value
        }

In [ ]:
from transformers import BertTokenizer

class Vectorizer:

    def __init__(self):
        self._tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased",
                                                        do_lower_case=False)

        self._label2class = label2class
        self._max_length = 134

    def vectorize(self, text: List[str], token_labels: List[str]) -> Tuple[List[str], List[int], List[int], List[int]]:
        tokenized_text, input_masks, labels = self._tokenize(text, token_labels)

        input_ids = self._tokenizer.convert_tokens_to_ids(tokenized_text) # Преобразует последовательность токенов в последовательность идентификаторов, используя словарь.

        tags = []
        for label in labels:
            tags.append(self._label2class[label])

        input_ids = self._pad(input_ids)
        input_masks = self._pad(input_masks)
        tags = self._pad(tags)

        return tokenized_text, input_ids, input_masks, tags

    def _pad(self, input: List[Any]) -> List[Any]:
        if len(input) >= self._max_length:
            print(len(input))
            return input[:self._max_length]
        while len(input) < self._max_length:
            input.append(0)
        return input

    def _tokenize(self, text: List[str], token_labels: List[str]) -> Tuple[List[str], List[int], List[str]]:
        tokenized_text = []
        labels = []

        for token, label in zip(text, token_labels):
            # Tokenize the word and count # of subwords the word is broken into
            tokenized_word = self._tokenizer.tokenize(token)
            n_subwords = len(tokenized_word)

            # Add the tokenized word to the final tokenized word list
            tokenized_text.extend(tokenized_word)

            # Add the same label to the new list of labels `n_subwords` times
            labels.extend([label] * n_subwords)

        try:

            inputs = self._tokenizer.encode_plus(
                tokenized_text,
                is_pretokenized=True,
                return_attention_mask=True,
                max_length=self._max_length,
                truncation=True
            )

        except:
            print(text)
            inputs = dict()
            inputs['attention_mask'] = np.zeros(self._max_length)

        return tokenized_text, inputs['attention_mask'], labels


In [ ]:
%pip install pymorphy2

In [ ]:
import pymorphy2
from pymorphy2.analyzer import Parse

In [ ]:
ADJF = 'ADJF'
CONJ = 'CONJ'
GRND = 'GRND'
NOUN = 'NOUN'
PREP = 'PREP'
PRTF = 'PRTF'
PRTS = 'PRTS'
VERB = 'VERB'

GENT = 'gent'


class HeuristicValidator:

    def __init__(self):
        self._morph = pymorphy2.MorphAnalyzer()

    def validate(self, result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        result = self._heuristic_1(result)
        result = self._heuristic_2(result)
        result = self._heuristic_3(result)
        result = self._heuristic_4(result)
        result = self._heuristic_5(result)
        result = self._heuristic_6(result)
        result = self._heuristic_7(result)
        # result = self._heuristic_8(result)
        result = self._heuristic_9(result)
        result = self._heuristic_10(result)
        return result

    def _heuristic_1(self, result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """ Валидация цепочек, которые представляют собой СУЩ + СУЩ в род.п., например: методы сжатия данных"""

        updated_result = {i: res for i, res in enumerate(result)}
        updated_tokens = set()

        for i, (result_pair_1, result_pair_2) in enumerate(zip(result, result[1:])):
            id_1 = i
            id_2 = i + 1
            # если последовательность не содержит терминов, то пропускаем
            if result_pair_1[1] == Tags.NOT_TERM.value and result_pair_2[1] == Tags.NOT_TERM.value:
                continue
            token_1, token_2 = result_pair_1[0], result_pair_2[0]
            pos_1 = self._morph.parse(token_1)[0].tag.POS
            case_2 = self._morph.parse(token_2)[0].tag.case
            if pos_1 in [NOUN, ADJF] and case_2 == GENT:
                if result_pair_1[1] not in TERM_SET and id_1 not in updated_tokens:
                    result_pair_1 = (token_1, Tags.B_TERM.value)
                    updated_result[id_1] = result_pair_1
                    updated_tokens.add(id_1)
                result_pair_2 = (token_2, Tags.I_TERM.value)
                updated_result[id_2] = result_pair_2
                updated_tokens.add(id_2)

        res = [updated_result[i] for i in range(len(result))]

        return res

    def _heuristic_2(self, result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """
        Если токены представляют собой последовательность ПРИЛ + СУЩ и оба помечены B-TERM, то приводим к
        последовательности B-TERM I-TERM
        """

        updated_result = {i: res for i, res in enumerate(result)}

        for i, (result_pair_1, result_pair_2) in enumerate(zip(result, result[1:])):
            id_1 = i
            id_2 = id_1 + 1
            # если последовательность не содержит терминов, то пропускаем
            if result_pair_1[1] == Tags.B_TERM.value and result_pair_2[1] == Tags.B_TERM.value:
                token_1, token_2 = result_pair_1[0], result_pair_2[0]
                parse_1 = self._morph.parse(token_1)
                parse_2 = self._morph.parse(token_2)
                is_adj_1 = self.__check_pos(ADJF, parse_1)
                is_noun_2 = self.__check_pos(NOUN, parse_2)
                if is_adj_1 and is_noun_2:
                    updated_result[id_1] = (token_1, Tags.B_TERM.value)
                    updated_result[id_2] = (token_2, Tags.I_TERM.value)

        res = [updated_result[i] for i in range(len(result))]

        return res

    def _heuristic_3(self, result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """ Удаление тэга B-TERM или I-TERM, если он был присовен токену знака пунктуации """

        updated_result = {i: res for i, res in enumerate(result)}

        for i, result_pair in enumerate(result):
            if result_pair[0] in ['.', ',', ':', ';'] and result_pair[1] in [Tags.B_TERM.value, Tags.I_TERM.value]:
                updated_result[i] = (result_pair[0], Tags.NOT_TERM.value)

        res = [updated_result[i] for i in range(len(result))]

        return res

    def _heuristic_4(self, result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """ Если последний токен в термине имеет часть речи ПРИЛ, а следующий токен - СУЩ, но либо не входит в термин,
        либо имеет тэг "B-TERM", то второй токен включаем в состав термина
        """

        updated_result = {i: res for i, res in enumerate(result)}

        for i, (result_pair_1, result_pair_2) in enumerate(zip(result, result[1:])):
            id_1 = i
            id_2 = id_1 + 1
            if result_pair_1[1] in TERM_SET and result_pair_2[1] != Tags.I_TERM.value:
                token_1, token_2 = result_pair_1[0], result_pair_2[0]
                parse_1 = self._morph.parse(token_1)
                parse_2 = self._morph.parse(token_2)
                is_adj_1 = self.__check_pos(ADJF, parse_1)
                is_prtf_1 = self.__check_pos(PRTF, parse_1)
                is_noun_2 = self.__check_pos(NOUN, parse_2)
                if is_noun_2:
                    if is_adj_1 or is_prtf_1:
                        updated_result[id_2] = (token_2, Tags.I_TERM.value)

        res = [updated_result[i] for i in range(len(result))]

        return res

    def _heuristic_5(self, result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """ Удаление тэга B-TERM у предлога и союза (допускаем, что предлог может входить в состав термина, но не может
        начинать его """

        updated_result = {i: res for i, res in enumerate(result)}

        for i, result_pair in enumerate(result):
            if result_pair[1] == Tags.B_TERM.value:
                parse = self._morph.parse(result_pair[0])
                is_prep = self.__check_pos(PREP, parse)
                is_conj = self.__check_pos(CONJ, parse)
                if is_prep or is_conj:
                    updated_result[i] = (result_pair[0], Tags.NOT_TERM.value)

        res = [updated_result[i] for i in range(len(result))]

        return res

    def _heuristic_6(self, result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """ Удаление тэга Термин у однозначного глагола или деепричастия """

        updated_result = {i: res for i, res in enumerate(result)}

        for i, result_pair in enumerate(result):
            if result_pair[1] in [Tags.B_TERM.value, Tags.I_TERM.value]:
                parse = self._morph.parse(result_pair[0])
                is_verb = self.__check_pos(VERB, parse)
                is_grnd = self.__check_pos(GRND, parse)
                is_prts = self.__check_pos(PRTS, parse)
                if len(parse) == 1:
                    if is_verb or is_grnd or is_prts:
                        updated_result[i] = (result_pair[0], Tags.NOT_TERM.value)

        res = [updated_result[i] for i in range(len(result))]

        return res

    def _heuristic_7(self, result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """ Если следующий за термином токен состоит только из латинских символов, то включаем его в состав термина """

        updated_result = {i: res for i, res in enumerate(result)}

        for i, (result_pair_1, result_pair_2) in enumerate(zip(result, result[1:])):
            id_1 = i
            id_2 = id_1 + 1
            if result_pair_1[1] in TERM_SET:
                token_2 = result_pair_2[0]
                is_latin = self._is_latin(token_2)
                if is_latin:
                    updated_result[id_2] = (token_2, Tags.I_TERM.value)

        res = [updated_result[i] for i in range(len(result))]

        return res

    def _heuristic_8(self, result: List[Tuple[str, str]]):
        """ Удаление тэга B-TERM у предлога и союза, которые идут после определения
        Н/р: ..исследование статистического и динамического провисания .."""
        pass

    def _heuristic_9(self, result: List[Tuple[str, str]]):
        """Чаще всего, всё, что стоит в кавычках, относится к термину, если он там выделен
        Н/р: .. компьютерный инструмент «оптимизация с ограничениями» .."""

        updated_result = {i: res for i, res in enumerate(result)}

        for i, result_pair in enumerate(result):
            if result_pair[0] in ['"', '«', '“', 'ˮ']:
                try:
                    if result[i + 1][1] == Tags.B_TERM.value:
                        is_fin = False
                        k = 2
                        while not is_fin:
                            next_pair = result[i + k]
                            is_fin = next_pair[0] in ['"', '»', '”', '‟']
                            if next_pair[1] not in TERM_SET and not is_fin:
                                updated_result[i + k] = (next_pair[0], Tags.I_TERM.value)
                            k += 1
                except IndexError:  # если кавычки стоят в конце текста
                    pass

        res = [updated_result[i] for i in range(len(result))]

        return res

    def _heuristic_10(self, result: List[Tuple[str, str]]):
        """Исправление разметки терминов, которые пишутся через дефис.
        Иногда часть составного термина при разметке "теряется"
        Н/р: ... курсов математической физики, физико-математической информатики и дифференциальных уравнений ..."""

        updated_result = {i: res for i, res in enumerate(result)}

        for i, result_pair in enumerate(result):
          try:
            if result_pair[0] == "-" and ((result[i+1][1] in TERM_SET) ^ (result[i-1][1] in TERM_SET)):
                if str(result[i-1][0])[-1] in ['о', 'е']:
                    if result[i-1][1] not in TERM_SET:
                        updated_result[i-1] = (result[i-1][0], Tags.B_TERM.value)
                    updated_result[i] = (result[i][0], Tags.I_TERM.value)
                    updated_result[i + 1] = (result[i + 1][0], Tags.I_TERM.value)
          except IndexError:
            pass

        res = [updated_result[i] for i in range(len(result))]

        return res

    def __check_pos(self, pos: str, parses: List[Parse]) -> bool:
            for parse in parses:
                if pos in parse.tag:
                    return True
            return False

    def _is_latin(self, token: str) -> bool:
        latin_symbols = 'qwertyuiopasdfghjklzxcvbnm'
        is_latin = True
        for char in token.lower():
            if char not in latin_symbols:
                is_latin = False
                return is_latin
        return is_latin


### DLExtractor

In [ ]:
class DLExtractor(BaseExtractor):
    """ Класс для извлечения терминов из текста с помощью модели """

    def __init__(self, weights_path):
        weights_path = weights_path
        self._model = get_model()
        self._model.load_weights(weights_path)
        self._vectorizer = Vectorizer()
        self._heuristic_validator = HeuristicValidator()
        self._class2label = class2label

    def extract(self, text: Union[str, List[str]]) -> List[Tuple[str, str]]:
        """ Извлечение терминов из входного текста

        :param text: входной текст, может быть строкой либо уже токенизированным (тогда списком строк)
        :return: Список кортежей, в которых первый элемент - токен, второй элемент - тэг
        """
        if isinstance(text, str): # проверка на то, что далее работаем со списком строк (токенов)
            tokens = tokenize(text)
        else:
            tokens = text

        labels = [Tags.NOT_TERM.value for i in range(len(tokens))]

        all_bpe_tokens = []
        all_predictions = []

        # делим список токенов на батчи, которые будут последовательно обрабатываться
        n_batches = int(len(tokens) / 50) + 1
        for i in range(n_batches):
            start = 50 * i
            end = min(len(tokens), 50 * i + 50)

            if start == end:
                break

            bpe_tokens, input_ids, input_masks, tags = self._vectorizer.vectorize(
                  tokens[start: end], labels[start: end]
              )
            preds = self._model.predict_on_batch([np.array([input_ids]), np.array([input_masks])])[0][0]
            all_bpe_tokens.extend(bpe_tokens)
            all_predictions.extend(preds[:len(bpe_tokens)])

        result = self._get_preds_with_tokens(all_bpe_tokens, all_predictions)
        result = self._heuristic_validator.validate(result)
        result = validate_sequence(result)

        if isinstance(text, list):
            result = self._align_tokens(text, result)

        return result

    def _align_tokens(self, input_tokens: List[str], result: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """ Выравнивание токенов
        В случаях, когда на вход пришёл уже токенизированный текст, токены в результирующем списке могут отличаться от
        тех, что в исходном из-за bpe-токенизации. Поэтому нужно выровнить результирующий список относительно входного,
        т.е. список токенов в обоих списках должен совпадать

        :param input_tokens: список токенов во входном списке
        :param result: результирующий список кортежей, в которых первый элемент - токен, второй - тэг
        :return: список кортежей, в которых первый элемент - токен, второй - тэг
        """
        # если списки токенов изначально совпадают, то сразу возвращаем результат
        resulted_tokens = [res[0] for res in result]
        if resulted_tokens == input_tokens:
            return result

        updated_result = []

        # фиксируем позицию токена в результирующем списке
        res_cursor = 0
        for i, token in enumerate(input_tokens):

            # токенизируем токен из входного списка. Если длина получившихся токенов == 1, то это не составной токен
            tokenized = tokenize(token)
            if len(tokenized) == 1: # если bpe-токен воспринимается как одно слово, добавляем его в результирующий список
              try:
                updated_result.append(result[i]) # result может быть длиннее, чем input_tokens
                res_cursor += 1
                continue
              except:
                print('here:', len(result))

            full_resulted = []
            tags = Counter()
            # собираем все токены в результирующем списке, которые лежат в промежутке от res_cursor до
            # res_cursor + количество токенов в tokenized
            for j in range(res_cursor, res_cursor + len(tokenized)):
                full_resulted.append(result[j][0])
                tags[result[j][1]] += 1

            # на случай, если составным токенам были присвоены разные тэги, то выберем тэг с максимальной частотой
            tag = tags.most_common()[0][0]
            updated_result.append((''.join(full_resulted), tag))

            # переведём позицию курсора на количество составных частей исходного токена
            res_cursor += len(tokenized)

        assert len(input_tokens) == len(updated_result), 'Alignment worked incorrect'

        return updated_result

    def _get_preds_with_tokens(self, bpe_tokens: List[str], preds) -> List[Tuple[str, str]]:
        """ Из предсказаний для bpe-токенов получаем предскания для целых токенов

        :param bpe_tokens: список bpe-токенов
        :param preds: список предиктов от модели
        :return: Список кортежей, в которых первый элемент - полноценный токен, второй элемент - тэг
        """
        result = []
        token = []
        tags = []

        for bpe_token, pred in zip(bpe_tokens, preds):

            # если bpe-токен не является началом целого токена, то он начинается с "##"
            if bpe_token.startswith('##'):
                token.append(bpe_token[2:])
                tags.append(self._class2label[np.argmax(pred)])

            else:
                # если уже собрали токен до этого, то обработаем его и положим в результирущий список
                if len(token) > 0:
                    self._process_token(result, tags, token)

                token = [bpe_token]
                tags = [self._class2label[np.argmax(pred)]]

        # обработаем последний токен и положим его в результирующий список
        self._process_token(result, tags, token)

        return result

    def _process_token(self, result: List[Tuple[str, str]], tags: List[str], token: List[str]):
        """Обработка токена: собираем его из bpe-токенов, выбираем нужный тэг

        :param result: результирующий список с токенами и тэгами
        :param tags: список тэгов, который был получен для составных bpe-токенов
        :param token: список bpe-токенов для данного токена
        """
        # объединяем составные bpe-токены в единую строку
        token_str = ''.join(token)

        tag = Tags.NOT_TERM.value

        # если во входном списке тэгов есть B-TERM или I-TERM, то выбираем данный тэг
        if Tags.B_TERM.value in tags:
            tag = Tags.B_TERM.value
        elif Tags.I_TERM.value in tags:
            tag = Tags.I_TERM.value

        result.append((token_str, tag))
    

## get new files

In [ ]:
class Marker:

    def __init__(self, path_to_weights, path_to_files):
        self._predictor = DLExtractor(path_to_weights)
        self._path_to_files = path_to_files
        self._max_len = 128

    def mark_dataset(self):
        c = 0
        dirs = sorted(os.listdir(self._path_to_files))
        path = self._path_to_files
        for article_dir in tqdm(dirs, desc='loading dataset'):
            if 'text.csv' in os.listdir(os.path.join(self._path_to_files, article_dir)):
                filename = os.path.join(self._path_to_files, article_dir, 'text.csv')
                df = pd.read_csv(filename)
                with open(filename, 'r') as f:
                    tokens = []
                    labels = []
                    file_samples = []
                    file_labels = []
                    reader = csv.DictReader(f)
                    for row in reader:
                        tokens.append(row['token'])
                        if len(tokens) == self._max_len:
                            file_samples.append(tokens)
                            preds = self._predictor.extract(tokens)
                            for (tok, tag) in preds:
                                labels.append(tag)
                            file_labels.append(labels)
                            tokens = []
                            labels = []
                    if len(tokens) > 0:
                        file_samples.append(tokens)
                        preds = self._predictor.extract(tokens)
                        for (tok, tag) in preds:
                            labels.append(tag)
                        file_labels.append(labels)
                        try:
                            df['tag'] = self.flatten_list(file_labels)
                        except:
                            print(file_labels)
                    df.to_csv(filename[:-4]+'_labeled.csv', index=False)
                c += 1
        if c == len(dirs)-1:
            print(f'{c} files marked up!')

    def flatten_list(self, lists):
      flat_list = []
      for l in lists:
          if type(l) is list:
              for item in l:
                  flat_list.append(item)
          else:
              flat_list.append(l)
      return flat_list


    def replace_space(self, y):
      for elem in y:
        if isinstance(elem, str):
          for i in range(len(y)):
            if y[i] == '':
              y[i] = 'O'
        elif isinstance(elem, list):
          for sample in y:
            for i in range(len(sample)):
              if sample[i] == '':
                sample[i] = 'O'
      return y


In [ ]:
if __name__ == '__main__':

    path_to_weights, path_to_files = 'weights/RUbert_cross_domain_main_weights.h5', 'Articles_dataset_rus'
    marker = Marker(path_to_weights, path_to_files)
    marker.mark_dataset()

*Note: Refactored styling mostly oriented on PEP8 with some custom addition to formatting from https://github.com/abramsci/seismology/blob/toolkit/toolkit/template.py*

**Workflow in a nutshell:**

0. Dataset loading and preprocessing

    0. Unzip the file located at DATASET_ZIP_PATH into INTERMID_DIR_PATH
    1. Iterate via 
    2. Validate tokens

1. Pretrained model loading and configuring



